In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib as plt
import torch.optim as optim

In [2]:
df = pd.read_parquet(r"C:\Users\adrie\OneDrive\Documents\EPFL\ML\Project 2\All_Relative_Results_Cleaned.parquet")

In [3]:
df.head(3)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
0,P04,Abduction,Correct,Frontal_Top,0.000,0.009328,0.779305,0.187880,0.020658,-0.693419,...,-0.633111,-0.054347,-0.511760,-0.185118,-0.068159,-0.602015,-0.598093,-0.066565,-0.600690,-0.592768
1,P04,Abduction,Correct,Frontal_Top,0.033,0.009353,0.779298,0.185050,0.021187,-0.693361,...,-0.630835,-0.054293,-0.511703,-0.184861,-0.068130,-0.602200,-0.596219,-0.066508,-0.600835,-0.590916
2,P04,Abduction,Correct,Frontal_Top,0.067,0.009722,0.779311,0.179212,0.021351,-0.693388,...,-0.630923,-0.054289,-0.511696,-0.184849,-0.067241,-0.601852,-0.596222,-0.065676,-0.600436,-0.590910


In [4]:
df['Camera'].unique()

array(['Frontal_Top', 'Frontal_Low', 'Side_Top'], dtype=object)

In [38]:
df_clean = df.dropna()
index = df_clean.columns.get_loc('time(s)')
df_right = df_clean.iloc[:, index+1:]
X = df_right
Y = df_clean['Exercise']

In [35]:
X.shape

(2183099, 99)

In [41]:
Y.shape

(2183099,)

**Model functions**

In [6]:
def get_model_weights(L, K, input_features, device):
    weights = []
    biases = []
    for idx in range(L):
        # Pour la première couche, utilisez le nombre de caractéristiques en entrée
        K_in = input_features if idx == 0 else K
        # Pour la dernière couche, définissez la sortie (par exemple 1 pour la régression)
        K_out = 1 if idx == L - 1 else K

        weight = torch.randn((K_in, K_out), device=device) * (2 / K_in) ** 0.5
        weight.requires_grad = True
        weights.append(weight)
        biases.append(torch.zeros(K_out, requires_grad=True, device=device))

    return weights, biases

In [7]:
def predict(X, weights, biases):
    assert len(weights) == len(biases)
    # ***************************************************
    for layer_idx in range(len(weights)):
        X = X @ weights[layer_idx] + biases[layer_idx]
        if layer_idx < len(weights) - 1:
            X = torch.clamp(X, min=0.0)
            # X = X * (X > 0) # Alternative implementation
    return X

In [8]:
def train_network(num_steps, weights, biases, X, Y_true, lr=1e-3, verbose=True):
    parameters = weights + biases
    optimizer = torch.optim.Adam(weights + biases, lr)
    losses = []
    for step in range(num_steps):
        Y = predict(X, weights, biases)
        loss = torch.mean((Y.flatten() - Y_true.flatten()) ** 2)
        loss.backward()
        # ***************************************************
        losses.append(loss.item())
        if verbose and step % 100 == 0:
            print(f"step={step} - loss={loss.item():0.4f}")
        optimizer.step()
        optimizer.zero_grad()

    if verbose:
        plt.plot(losses)
        plt.xlabel("Step")
        plt.ylabel("Loss")
        plt.show()

**Model visualization** 

In [9]:
def plot_heatmap(X, Y, N_sqrt, mark_level=None):
    x1_grid = X[:, 0].reshape((N_sqrt, N_sqrt))
    x2_grid = X[:, 1].reshape((N_sqrt, N_sqrt))
    plt.pcolormesh(x1_grid, x2_grid, Y.reshape_as(x1_grid), cmap="bwr", vmin=-1, vmax=1)
    plt.axis([x1_grid.min(), x1_grid.max(), x2_grid.min(), x2_grid.max()])
    plt.colorbar()
    if mark_level is None:
        mark_level = float(Y.mean())
    if isinstance(mark_level, (int, float)):
        plt.contour(
            x1_grid,
            x2_grid,
            Y.reshape_as(x1_grid),
            levels=[mark_level],
            colors="k",
            linewidths=1,
        )
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")


def plot_cross_section(X, Y, value=0, **kwargs):
    x2 = X[:, 1]
    value_rounded = x2[torch.argmin(torch.abs(x2 - value))]
    mask = torch.isclose(X[:, 1], value_rounded)
    x1 = X[mask, 0]
    y = Y[mask]
    plt.plot(x1, y, **kwargs)

*First try*

In [42]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

In [43]:
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(Y_encoded, dtype=torch.float32)

In [44]:
torch.isnan(X_tensor)

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [45]:
from sklearn.model_selection import train_test_split

# Supposons que X_tensor et Y_tensor sont vos données complètes
X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)  # 20% des données pour le test


In [46]:
from torch.utils.data import TensorDataset, DataLoader

# Créer des jeux de données pour l'entraînement et le test
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

# Créer des DataLoader pour l'entraînement et le test
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [47]:
torch.isnan(X_train).any()

tensor(False)

In [48]:
N_sqrt = 100
D = 4
H = 32
nbr_input_features = 99
# Alternative might require more steps, a wider network or other learning rates

# You can set this to 'cuda' if you have a GPU available.
# In this case the dataset and model is so small that we only expect a minimal speed difference.
device = torch.device("cpu")
steps = 1000
lr = 1e-3

# Create model
weights, biases = get_model_weights(D, H,nbr_input_features, device)
assert all(
    [p.requires_grad for p in weights + biases]
), "All model parameters should have requires_grad=True"
assert all(
    [p.is_leaf for p in weights + biases]
), "All model parameters must be leaf tensors"

# Train the model
train_network(steps, weights, biases, X_train, Y_train, lr)

# Compute the final model predictions
# Typically this would be on some sort of validation or test data
with torch.no_grad():
    Y_model = predict(X_train, weights, biases)

step=0 - loss=12.8862
step=100 - loss=1.0795


KeyboardInterrupt: 

**Model prep**

In [70]:
Y_encoded   

array([0, 0, 0, ..., 6, 6, 6])

In [64]:
X_train

tensor([[ 9.3330e+00, -6.6489e-01,  2.1861e-01,  ...,  9.4369e-01,
          1.2564e-01, -1.9798e-01],
        [ 1.9667e+01,  3.5258e-01,  4.8346e-02,  ..., -1.5303e-02,
          1.0006e-01, -3.0847e-01],
        [ 3.0200e+01,  2.0821e-01,  1.5905e-01,  ..., -2.5079e-01,
          8.9252e-02, -7.0971e-01],
        ...,
        [ 1.2900e+01, -5.5064e-03,  7.7270e-01,  ..., -1.7922e-01,
          1.1402e-02, -2.8603e-01],
        [ 3.1367e+01, -7.2049e-01,  1.3203e-01,  ...,  9.9081e-01,
          3.4162e-02, -1.7648e-01],
        [ 2.3000e+00,  1.5152e-01,  4.6636e-01,  ..., -2.8823e-01,
          8.4477e-02, -6.6218e-02]])

In [30]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(X.shape[1], 64)  # X.shape[1] est le nombre de caractéristiques
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 7)  # Supposons 7 classes pour Y
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.softmax(x)
        return x

model = NeuralNetwork()


In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [67]:
train_loader

In [66]:
# Assurez-vous que Y_tensor est de type long
num_epochs = 5
Y_train = Y_train.long()
X_train = X_train.long()
# Puis créez vos jeux de données et vos DataLoaders
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ... (reste de votre code pour la définition du modèle, etc.)

# Entraînement
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float

In [ ]:
# Assurez-vous de désactiver le calcul du gradient pour l'évaluation
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:  # Supposant que vous avez un test_loader
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')
